In [1]:
import logging
import queue
import random
import threading
import time

import flask
import numpy as np
from bokeh.io import output_notebook, push_notebook, show
from bokeh.models import AjaxDataSource, ColumnDataSource, CustomJS
from bokeh.plotting import figure, show
from flask import Flask, jsonify, make_response, request
from flask.logging import default_handler
from ipywidgets import interact
from werkzeug.serving import make_server

In [2]:
logging.getLogger("werkzeug").setLevel(logging.WARNING)

In [3]:
class ServerThread(threading.Thread):
    # Credit: https://stackoverflow.com/a/45017691/2063031

    def __init__(self, app, host="localhost", port=0, threaded=False):
        super(ServerThread, self).__init__()
        self.server = make_server(host=host, port=port, app=app, threaded=threaded)
        self.context = app.app_context()
        self.context.push()

    def run(self):
        self.server.serve_forever()

    def stop(self):
        self.server.shutdown()

    def __enter__(self):
        self.start()
        return self

    def __exit__(self, *args):
        self.stop()

In [4]:
def create_data_publisher_app(data_source, processor_fn=None, timeout=1):
    """Create a flask app that can serve data from the data_source.

    Args:
        data_source (queue.Queue): A source of data conforming to the queue.Queue API.
            WARNING: It is probably a good idea if data_source is thread-safe.
        processor_fn (callable): A function that will be applied to the data before serving.
        timeout (int): Number of seconds to wait while pulling data from queue.
    """
    from flask_cors import cross_origin
    from flask_cors import CORS

    app = flask.Flask(__name__)
    CORS(app, origins="*", allow_headers=[
        "Content-Type", "Authorization", "Access-Control-Allow-Credentials"],
        supports_credentials=True, intercept_exceptions=False)

    # TODO(ostrokach): This seems to be necessary, but not sure why.
    def crossdomain(f):
        def wrapped_function(*args, **kwargs):
            resp = make_response(f(*args, **kwargs))
            h = resp.headers
            h["Access-Control-Allow-Origin"] = "*"
            h["Access-Control-Allow-Methods"] = "GET, OPTIONS, POST"
            h["Access-Control-Max-Age"] = str(21600)
            requested_headers = request.headers.get("Access-Control-Request-Headers")
            if requested_headers:
                h["Access-Control-Allow-Headers"] = requested_headers
            else:
                h["Access-Control-Allow-Headers"] = "Content-Type,Authorization"
            h["Access-Control-Allow-Credentials"] = "true"
            
            return resp

        return wrapped_function

    @app.route("/data", methods=["GET", "OPTIONS", "POST"])
    #@cross_origin()
    @crossdomain
    def data():
        data = None
        try:
            data = data_source.get(timeout=0.1)
        except queue.Empty:
            pass
        else:
            if processor_fn:
                data = processor_fn(data)
        return jsonify([data] if data is not None else [])

    return app

In [5]:
def generate_plot(data_url):
    adapter = CustomJS(
        code="""
        const result = {x: [], y: []};
        const pts = cb_data.response;
        for (i=0; i<pts.length; i++) {
            result.x.push(pts[i][0])
            result.y.push(pts[i][1])
        }
        return result;
    """
    )

    source = AjaxDataSource(
        data_url=data_url, polling_interval=100, adapter=adapter, mode="append"
    )

    p = figure(
        plot_height=300,
        plot_width=800,
        background_fill_color="lightgrey",
        title="Streaming Noisy sin(x) via Ajax",
    )
    p.circle("x", "y", source=source)

    p.x_range.follow = "end"
    p.x_range.follow_interval = 10

    return p


# show(generate_plot(None))

In [6]:
data_queue = queue.Queue()

x = 0
y = np.sin(x) + np.random.random()

data_queue.put((x, y))

for i in range(10000):
    x += 0.1
    y = np.sin(x) + np.random.random()
    data_queue.put((x, y))

In [7]:
endpoint = ServerThread(create_data_publisher_app(data_queue), host="localhost", port=0, threaded=False)

In [8]:
endpoint.start()

In [9]:
data_url = "http://localhost:{}/data".format(endpoint.server.port)
print(data_url)

http://localhost:41953/data


In [10]:
output_notebook()

show(generate_plot(data_url))

Loading BokehJS ...

In [11]:
data_queue2 = queue.Queue()

x = 0
y = np.sin(x) + np.random.random()

data_queue2.put((x, y))

for i in range(10000):
    x += 0.1
    y = np.sin(x) + np.random.random()
    data_queue2.put((x, y))

In [12]:
endpoint2 = ServerThread(create_data_publisher_app(data_queue2), host="localhost", port=0, threaded=False)

In [13]:
endpoint2.start()

In [14]:
data_url = "http://localhost:{}/data".format(endpoint2.server.port)
print(data_url)

http://localhost:43257/data


In [15]:
output_notebook()

show(generate_plot(data_url))

Loading BokehJS ...